In [2]:
import polars as pl
import pandas as pd
from tqdm import tqdm
import gc

lazy_df = pl.scan_parquet('datasets/train.parquet')

unique_date_ids = lazy_df.select(pl.col("date_id").unique()).collect().to_series().to_list()

df = pd.DataFrame()

batch_size = 10
for i in tqdm(range(0, len(unique_date_ids[-500:]), batch_size), desc="Processing batches"):
    batch_date_ids = unique_date_ids[-500:][i:i + batch_size]
    df_chunk = lazy_df.filter(pl.col("date_id").is_in(batch_date_ids)).collect().to_pandas()
    df = pd.concat([df, df_chunk], ignore_index=True)
    del df_chunk
    gc.collect() 

Processing batches:   0%|          | 0/50 [00:00<?, ?it/s]


PanicException: called `Result::unwrap()` on an `Err` value: ComputeError(ErrString("RecordBatch requires all its arrays to have an equal number of rows"))